# Coin Detection

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.cmap'] = 'gray'

In [ ]:
# Image path
imagePath = "images/CoinsA.png"
# Read image
# Store it in the variable image

image = cv2.imread(imagePath, cv2.IMREAD_COLOR)
imageCopy = image.copy()
plt.imshow(image[:,:,::-1]);
plt.title("Original Image")

## <font style = "color:rgb(50,120,229)">Step 2.1: Convert Image to Grayscale</font>

In [ ]:
# Convert image to grayscale
# Store it in the variable imageGray

imageGray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(121)
plt.imshow(image[:,:,::-1]);
plt.title("Original Image")
plt.subplot(122)
plt.imshow(imageGray);
plt.title("Grayscale Image");

## <font style = "color:rgb(50,120,229)">Step 2.2: Split Image into R,G,B Channels</font>

In [ ]:
# Split cell into channels
# Store them in variables imageB, imageG, imageR

imageB, imageG, imageR = cv2.split(image)

In [ ]:
plt.figure(figsize=(20,12))
plt.subplot(141)
plt.imshow(image[:,:,::-1]);
plt.title("Original Image")
plt.subplot(142)
plt.imshow(imageB);
plt.title("Blue Channel")
plt.subplot(143)
plt.imshow(imageG);
plt.title("Green Channel")
plt.subplot(144)
plt.imshow(imageR);
plt.title("Red Channel");

## <font style = "color:rgb(50,120,229)">Step 3.1: Perform Thresholding</font>

In [ ]:

_, imageBThres = cv2.threshold(imageB, 55, 255, cv2.THRESH_BINARY_INV)
_, imageGThres = cv2.threshold(imageG, 55, 255, cv2.THRESH_BINARY_INV)
_, imageRThres = cv2.threshold(imageR, 55, 255, cv2.THRESH_BINARY_INV)

plt.subplot(131); plt.title("Thresholded B channel"); plt.imshow(imageBThres)
plt.subplot(132); plt.title("Thresholded G channel"); plt.imshow(imageGThres)
plt.subplot(133); plt.title("Thresholded R channel"); plt.imshow(imageRThres)

# setting thres image to green channel as it has the best contrast
imageThres = imageGThres

In [ ]:
# Display the thresholded image

plt.title("Final thresholded image")
plt.imshow(imageThres)

## <font style = "color:rgb(50,120,229)">Step 3.2: Perform morphological operations</font>

In [ ]:

element = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))

imageMorph = cv2.morphologyEx(imageThres, cv2.MORPH_OPEN, element, iterations=2)
plt.title("Image after opening")
plt.imshow(imageMorph)

In [ ]:

# imageMorph = cv2.dilate(imageMorph, element, iterations=2)
# imageMorph = cv2.erode(imageMorph, element, iterations=2)

imageMorph = cv2.morphologyEx(imageThres, cv2.MORPH_CLOSE, element, iterations=2)
plt.title("Image after closing")
plt.imshow(imageMorph)

In [ ]:
# Display all the images
# you have obtained in the intermediate steps

imageMorph = cv2.morphologyEx(imageMorph, cv2.MORPH_OPEN, element, iterations=3)
plt.title("Image after opening again")
plt.imshow(imageMorph)

In [ ]:
# Get structuring element/kernel which will be used for dilation

element2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
element3 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))


In [ ]:

imageMorph = imageThres.copy()

imageMorph = cv2.morphologyEx(imageMorph, cv2.MORPH_OPEN, element2, iterations=2)
imageMorph = cv2.dilate(imageMorph, element2, iterations=1)
imageMorph = cv2.morphologyEx(imageMorph, cv2.MORPH_CLOSE, element2, iterations=3)
imageMorph = cv2.erode(imageMorph, element2, iterations=1)
plt.title("Image after series of morph operations")
plt.imshow(imageMorph)

In [ ]:
_, imageMorphInv = cv2.threshold(imageMorph, 100, 255, cv2.THRESH_BINARY_INV)
plt.imshow(imageMorphInv)

## <font style = "color:rgb(50,120,229)">Step 4.1: Create SimpleBlobDetector</font>

In [ ]:
# Set up the SimpleBlobdetector with default parameters.
params = cv2.SimpleBlobDetector_Params()

params.blobColor = 0

params.minDistBetweenBlobs = 2

# Filter by Area.
params.filterByArea = False

# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.8

# Filter by Convexity
params.filterByConvexity = True
params.minConvexity = 0.8

# Filter by Inertia
params.filterByInertia =True
params.minInertiaRatio = 0.8

In [ ]:
# Create SimpleBlobDetector
detector = cv2.SimpleBlobDetector_create(params)

## <font style = "color:rgb(50,120,229)">Step 4.2: Detect Coins</font>

In [ ]:
# Detect blobs

keypoints = detector.detect(imageMorph)

In [ ]:
# Print number of coins detected

print(f"Number of blobs detected: {len(keypoints)}")

**Note that we were able to detect all 9 coins. So, that's your benchmark.**

In [ ]:
# Mark coins using image annotation concepts we have studied so far

im = image.copy()

for k in keypoints:
    x, y = k.pt
    x = int(x)
    y = int(y)
    r = int(k.size // 2)
    cv2.circle(im, (x, y), 2, (255, 0, 0), -1)
    cv2.circle(im, (x, y), r, (0, 255, 0), 2)

In [ ]:
# Display the final image


plt.imshow(im[..., ::-1])

## <font style = "color:rgb(50,120,229)">Step 4.4: Perform Connected Component Analysis</font>

In [ ]:
def displayConnectedComponents(im):
    imLabels = im
    # The following line finds the min and max pixel values
    # and their locations in an image.
    (minVal, maxVal, minLoc, maxLoc) = cv2.minMaxLoc(imLabels)
    # Normalize the image so the min value is 0 and max value is 255.
    imLabels = 255 * (imLabels - minVal)/(maxVal-minVal)
    # Convert image to 8-bits unsigned type
    imLabels = np.uint8(imLabels)
    # Apply a color map
    imColorMap = cv2.applyColorMap(imLabels, cv2.COLORMAP_JET)
    # Display colormapped labels
    plt.imshow(imColorMap[:,:,::-1])

In [ ]:
# Find connected components

_, imComponents = cv2.connectedComponents(imageMorphInv)

In [ ]:
# Print number of connected components detected

print(f"Number of connected components detected = {imComponents.max()}")

In [ ]:
# Display connected components using displayConnectedComponents
# function

displayConnectedComponents(imComponents)

## <font style = "color:rgb(50,120,229)">Step 4.5: Detect coins using Contour Detection</font>

In [ ]:
# Find all contours in the image

contours, hierarchy = cv2.findContours(imageMorph, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
# Print the number of contours found

print(f"Number of contours found = {len(contours)}")

In [ ]:
# Draw all contours

im2 = image.copy()
cv2.drawContours(im2, contours, -1, (0,255,255), 3, hierarchy=hierarchy)
plt.title("Image with all contours")
plt.imshow(im2[..., ::-1])

Let's only consider the outer contours.

In [ ]:
# Remove the inner contours
# Display the result

ext_contours = [contours[i] for i, h in enumerate(hierarchy[0]) if h[3] == -1]
print(f"Number of outer contours {len(ext_contours)}")

So, we only need the inner contours. The easiest way to do that will be to remove the outer contour using area.

In [ ]:
# Print area and perimeter of all contours

areas = []
for i, c in enumerate(contours):
    area = cv2.contourArea(c)
    areas.append(area)
    print(f"Contour #{i+1}  has area {area} and perimeter = {cv2.arcLength(c, True)}")

In [ ]:
# Print maximum area of contour
# This will be the box that we want to remove

print(f"Maximum area of the contour is {max(areas)}")

In [ ]:
# Remove this contour and plot others

int_contours = contours[1:]
im3 = image.copy()
cv2.drawContours(im3, int_contours, -1, (0,255,255), 3)
plt.title("Image with contours")
plt.imshow(im3[..., ::-1])

In [ ]:
# Fit circles on coins

im4 = image.copy()

for c in int_contours:
    center, radius = cv2.minEnclosingCircle(c)
    
    cv2.circle(im4, (int(center[0]),int(center[1])), int(radius), (255, 255, 0), 3)

plt.title("Image with enclosing circles")
plt.imshow(im4[..., ::-1])
